In [1]:
import os
import numpy as np
from numpy import array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations
from PIL import Image

In [2]:
#Now we will build the neural network model
model = Sequential()

#Adding the input layer
model.add(layers.Dense(units=1024,input_shape=(1,1,1024)))

#Adding the first hidden layer
model.add(layers.Dense(units=1024, activation='tanh'))

#Adding the second hidden layer
model.add(layers.Dense(units=512, activation='sigmoid'))

#Adding the third hidden layer
model.add(layers.Dense(units=100, activation='relu'))

#Adding the output layer
model.add(layers.Dense(units=2, activation='softmax'))

In [10]:
#flatten each image and put them into a list
flattened_images = []
for i in range(1, 101):
    f_string = 'train/' + str(i) + '.jpg'
    im = Image.open(f_string)
    im = np.array(im)
    flat_im = im.flatten()
    flattened_images.append(flat_im)

print('old flatten: \n', flattened_images[0])
for img in flattened_images:
    minimum_val = img.min()
    maximum_val = img.max()
    print(minimum_val)
    print(maximum_val)
    for i in range(0, len(img)):
        img[i] = (img[i] - minimum_val)/(maximum_val - minimum_val)
    break

print('new flatten: \n', flattened_images[0])
        

old flatten: 
 [ 1 12 16 ... 23 17 20]
0
93
new flatten: 
 [0 0 0 ... 0 0 0]
